In [ ]:
#imports
from matplotlib import pyplot as plt
from tensorflow import keras
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix, precision_recall_fscore_support

In [ ]:
#load SMHD files. Replace filenames. 
with open('evaluation-x-smhd', 'rb') as file:
    evaluation_x = pickle.load(file)

with open('evaluation-y-smhd', 'rb') as file:
    evaluation_y = pickle.load(file)

with open('x-training-smhd', 'rb') as file:
    X_training = pickle.load(file)

with open('y-training-smhd', 'rb') as file:
    y_training = pickle.load(file) 

In [ ]:
#making the arrays into lists to use as input in model
x_train_array= np.array(X_training.values.tolist())
x_test_array= np.array(evaluation_x.values.tolist())
y_train_array= np.array(y_training.values.tolist())
y_test_array= np.array(evaluation_y.values.tolist())

In [ ]:
#creating the model with layers
input_shape = (1000,)

ann_model = Sequential([
    Dense(1000, input_shape=input_shape, activation='relu'),
    Dense(500, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax'),
])

In [ ]:
#compiling the model and setting the learning rate and loss function
ann_model.compile(optimizer=Adam(learning_rate=0.005), 
              loss=keras.losses.SparseCategoricalCrossentropy(), 
              metrics=['accuracy']) #calculating accuracy during training
  
#fitting the model with training data. 
ann_model.fit(x_train_array, y_train_array, epochs=10, batch_size=32, validation_data=(x_test_array, y_test_array))

In [ ]:
#evaluating model
ann_model.evaluate(evaluation_x, evaluation_y, batch_size=64)

In [ ]:
#evaluating predictions on test dataset

predictions = ann_model.predict(x_test_array)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test_array, predicted_labels)
precision, recall1, f1, _ = precision_recall_fscore_support(y_test_array, predicted_labels, average='weighted')

print('Accuracy:', accuracy)
print(f'Precision: {precision:.2f}')
print('Recall1:', recall1)
print(f'F1 Score: {f1:.2f}')

#creating the confusion matrix
cm = confusion_matrix(y_test_array, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_test_array))
disp.plot(cmap='Blues', values_format='d')
plt.show()
